In [1]:
from IPython.display import clear_output

!pip install transformers

import os
import math
import datetime as dt

import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset

from tqdm.notebook import tqdm

from transformers import BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification

clear_output()

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Studium/5. Semester/NLP/Trainingsdatensatz - Tabellenblatt2.csv')
df.sort_values(by='Classes', inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Questions,Classes
0,Bye Bye,bye
1,"Ok great, thank you",bye
2,Ok bye!,bye
3,"Great, thanks!",bye
4,goodbye,bye


In [3]:
le = LabelEncoder()
df['label'] = le.fit_transform(df.Classes.values)

possible_labels = df.Classes.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
print(label_dict)

df.head()

{'bye': 0, 'cloud': 1, 'general_weather': 2, 'greeting': 3, 'rain': 4, 'sun_hours': 5, 'temperature': 6, 'unknown': 7, 'wind': 8}


,Questions,Classes,label
0,Bye Bye,bye,0
1,"Ok great, thank you",bye,0
2,Ok bye!,bye,0
3,"Great, thanks!",bye,0
4,goodbye,bye,0


In [4]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Classes', 'label', 'data_type']).count()

Questions
Classes         label data_type           
bye             0     train             45
                      val               15
cloud           1     train            105
                      val               35
general_weather 2     train            231
                      val               77
greeting        3     train             72
                      val               24
rain            4     train            120
                      val               40
sun_hours       5     train             63
                      val               21
temperature     6     train            123
                      val               41
unknown         7     train             42
                      val               14
wind            8     train            108
                      val               36

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
clear_output()

In [6]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Questions.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Questions.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [7]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [8]:
len(dataset_train), len(dataset_val)

(909, 303)

In [9]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [11]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [12]:
epochs = 7

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [13]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [14]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [16]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/Studium/5. Semester/NLP/Models/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/303 [00:00<?, ?it/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Studium/5. Semester/NLP/Models/finetuned_BERT_epoch_7.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)
#predictions

In [ ]:
accuracy_per_class(predictions, true_vals)

#GET PREDICTIONS

In [ ]:
bert = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
bert.load_state_dict(torch.load('/content/drive/MyDrive/Studium/5. Semester/NLP/Models/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

clear_output()

In [ ]:
question = ['Will it rain',
            'Do I need an umbrella',
            'What is the temperature',
            'Is it cold',
            'bye',
            'hello',
            'Is it windy outside',
            'Will the weather be good today',
            'When does the sun set?']

data = {'Question': question}
df = pd.DataFrame(data = data)
value_to_predict = df.Question.values

encoded_data = tokenizer.batch_encode_plus(
    value_to_predict, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_id = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
value_to_predict_tensor = TensorDataset(input_id, attention_mask)

dataloader_prediction = DataLoader(value_to_predict_tensor, 
                                   sampler=SequentialSampler(value_to_predict_tensor), 
                                   batch_size=1)
bert.eval()
    
prediction = []
    
for batch in dataloader_prediction:
    
  batch = tuple(b.to(device) for b in batch)
#  print('Batch: \n', batch)
  inputs = {'input_ids':      batch[0],
            'attention_mask': batch[1]
            }
#  print('Inputs: \n',inputs)

  with torch.no_grad():        
    outputs = bert(**inputs)
#    print(outputs)
  
  logits = outputs[0]
#  print(logits)
  logits = logits.detach().cpu().numpy()
#  print(logits)
  prediction.append(logits)
#  print(prediction)

prediction = np.concatenate(prediction, axis=0)
#  print(prediction)

label_dict_inverse = {v: k for k, v in label_dict.items()}

prediction_flat = np.argmax(prediction, axis=1).flatten()
print(question)
print(label_dict)
prediction_flat